In [5]:
from mmcv.runner import get_dist_info, init_dist, load_checkpoint
from pyskl.models import build_model
from pyskl.datasets import build_dataloader, build_dataset

import mmcv
import os
import os.path as osp
import time
import torch
import torch.distributed as dist
from mmcv import Config, load
from mmcv.cnn import fuse_conv_bn
from mmcv.engine import multi_gpu_test
from mmcv.fileio.io import file_handlers
from mmcv.parallel import MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

from pyskl.datasets import build_dataloader, build_dataset
from pyskl.models import build_model
from pyskl.utils import cache_checkpoint, mc_off, mc_on, test_port

In [6]:
config_file = "D:\\FYP\\HAR-ZSL-XAI\\pyskl\\configs\\posec3d\\slowonly_r50_346_k400\\limb.py"
cfg = Config.fromfile(config_file)

## Load model

In [7]:
checkpoint_path = "../model_saves/Pose3D/kinetic_400_346_limb.pth"

In [8]:
model = build_model(cfg.model)
load_checkpoint(model, checkpoint_path, map_location='cpu')

load checkpoint from local path: ../model_saves/Pose3D/kinetic_400_346_limb.pth


OrderedDict([('backbone.conv1.conv.weight',
              tensor([[[[[-1.2584e-02, -8.3466e-04, -6.1006e-03,  ..., -8.4638e-03,
                          -2.8386e-02, -1.5192e-02],
                         [ 3.3752e-02,  1.5777e-02,  3.9949e-02,  ..., -1.9431e-02,
                           1.1950e-03,  2.3671e-02],
                         [-2.6003e-02, -1.8944e-03,  6.0830e-02,  ..., -3.8321e-02,
                          -1.7503e-02, -1.4342e-02],
                         ...,
                         [-2.6298e-02, -3.7876e-02,  8.3003e-03,  ..., -4.1446e-03,
                          -3.4105e-02, -8.6852e-04],
                         [-1.4345e-02, -3.2849e-02, -6.6792e-02,  ..., -4.1586e-02,
                           1.2901e-02, -2.5720e-03],
                         [ 1.9790e-04, -6.3391e-02, -1.0212e-01,  ...,  4.8647e-02,
                           6.3595e-02,  1.0857e-02]]],
              
              
                       [[[-1.5862e-02, -2.1877e-02, -2.6346e-02,  ...,  

In [9]:
model

Recognizer3D(
  (backbone): ResNet3dSlowOnly(
    (conv1): ConvModule(
      (conv): Conv3d(17, 32, kernel_size=(1, 7, 7), stride=(1, 1, 1), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck3d(
        (conv1): ConvModule(
          (conv): Conv3d(32, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
          (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=T

## Pretrained model inference

In [10]:
out = "Pose3DResults"

eval_cfg = cfg.get('evaluation', {})
keys = ['interval', 'tmpdir', 'start', 'save_best', 'rule', 'by_epoch', 'broadcast_bn_buffers']
for key in keys:
    eval_cfg.pop(key, None)

eval_cfg['metrics'] = ['top_k_accuracy', 'mean_class_accuracy']
os.makedirs(out,exist_ok=True)

# set cudnn benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.data.test.test_mode = True

if not hasattr(cfg, 'dist_params'):
    cfg.dist_params = dict(backend='nccl')

default_mc_cfg = ('localhost', 22077)
memcached = cfg.get('memcached', False)

In [12]:
def inference_pytorch(args, cfg, data_loader):
    """Get predictions by pytorch models."""
    if args.average_clips is not None:
        # You can set average_clips during testing, it will override the
        # original setting
        if cfg.model.get('test_cfg') is None and cfg.get('test_cfg') is None:
            cfg.model.setdefault('test_cfg',
                                 dict(average_clips=args.average_clips))
        else:
            if cfg.model.get('test_cfg') is not None:
                cfg.model.test_cfg.average_clips = args.average_clips
            else:
                cfg.test_cfg.average_clips = args.average_clips

    # build the model and load checkpoint
    model = build_model(cfg.model)

    if args.checkpoint is None:
        work_dir = cfg.work_dir
        args.checkpoint = osp.join(work_dir, 'latest.pth')
        assert osp.exists(args.checkpoint)

    args.checkpoint = cache_checkpoint(args.checkpoint)
    load_checkpoint(model, args.checkpoint, map_location='cpu')

    if args.fuse_conv_bn:
        model = fuse_conv_bn(model)

    model = MMDistributedDataParallel(
        model.cuda(),
        device_ids=[torch.cuda.current_device()],
        broadcast_buffers=False)
    outputs = multi_gpu_test(model, data_loader, args.tmpdir)

    return outputs

In [13]:
init_dist('pytorch', **cfg.dist_params)
rank, world_size = get_dist_info()
cfg.gpu_ids = range(world_size)

KeyError: 'RANK'

In [ ]:
# build the dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
dataloader_setting = dict(
    videos_per_gpu=cfg.data.get('videos_per_gpu', 1),
    workers_per_gpu=cfg.data.get('workers_per_gpu', 1),
    shuffle=False)
dataloader_setting = dict(dataloader_setting, **cfg.data.get('test_dataloader', {}))
data_loader = build_dataloader(dataset, **dataloader_setting)

In [ ]:
if rank == 0 and memcached:
    # mc_list is a list of pickle files you want to cache in memory.
    # Basically, each pickle file is a dictionary.
    mc_cfg = cfg.get('mc_cfg', default_mc_cfg)
    assert isinstance(mc_cfg, tuple) and mc_cfg[0] == 'localhost'
    if not test_port(mc_cfg[0], mc_cfg[1]):
        mc_on(port=mc_cfg[1], launcher=args.launcher)
    retry = 3
    while not test_port(mc_cfg[0], mc_cfg[1]) and retry > 0:
        time.sleep(5)
        retry -= 1
    assert retry >= 0, 'Failed to launch memcached. '

dist.barrier()
outputs = inference_pytorch(args, cfg, data_loader)

load checkpoint from local path: ../data/limb_nturgb_120_xsub.pth


OrderedDict([('backbone.conv1.conv.weight',
              tensor([[[[[-4.2655e-02, -1.8377e-02, -2.2783e-03,  ..., -1.2765e-02,
                          -3.5315e-03, -2.3672e-03],
                         [-6.4037e-02, -3.0066e-02, -2.0834e-02,  ...,  8.5235e-03,
                          -1.1630e-02, -1.1425e-03],
                         [ 7.7105e-03,  4.8927e-02, -1.5288e-03,  ..., -5.1428e-02,
                          -1.7376e-02,  2.2314e-02],
                         ...,
                         [ 1.3193e-02, -6.4285e-03,  6.0435e-03,  ...,  2.9285e-02,
                           1.0417e-02, -2.5427e-02],
                         [-1.9435e-02, -3.9191e-02,  1.1970e-03,  ...,  4.5495e-02,
                          -8.3867e-03, -5.2482e-02],
                         [-1.4089e-02, -2.8624e-02,  1.0591e-02,  ...,  2.7153e-03,
                           5.2541e-03, -5.6652e-02]]],
              
              
                       [[[-2.7352e-02, -1.6306e-02, -1.0725e-02,  ..., -

Recognizer3D(
  (backbone): ResNet3dSlowOnly(
    (conv1): ConvModule(
      (conv): Conv3d(17, 32, kernel_size=(1, 7, 7), stride=(1, 1, 1), padding=(0, 3, 3), bias=False)
      (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck3d(
        (conv1): ConvModule(
          (conv): Conv3d(32, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
          (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): ReLU(inplace=T

In [7]:
dataset = build_dataset(data_cfg.test, dict(test_mode=True))
dataloader_setting = dict(
    videos_per_gpu=data_cfg.get('videos_per_gpu', 1),
    workers_per_gpu=data_cfg.get('workers_per_gpu', 1),
    shuffle=False)
dataloader_setting = dict(dataloader_setting, **data_cfg.get('test_dataloader', {}))
data_loader = build_dataloader(dataset, **dataloader_setting)

AttributeError: 'dict' object has no attribute 'test'